# Topic 3: Huấn luyện mô hình ML
Công ty C muốn sử dụng ML để đánh giá ý định mua sắm của khác hàng dựa vào các dữ liệu đã thu thập được.
Dữ liệu: https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset
Hãy huấn luyện mô hình ML để đạt được dự đoán F1-score là tốt nhất có thể.
Yêu cầu:
- Đánh giá kết quả trung bình F1 score trên bộ dữ liệu trên với KFOLD (K=5)
- Sử dụng tối thiểu 3 mô hình khác nhau để so sánh kết quả trung bình F1-score


In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
online_shoppers_purchasing_intention_dataset = fetch_ucirepo(id=468)

# data (as pandas dataframes)
X = online_shoppers_purchasing_intention_dataset.data.features
y = online_shoppers_purchasing_intention_dataset.data.targets

# metadata
print(online_shoppers_purchasing_intention_dataset.metadata)

# variable information
print(online_shoppers_purchasing_intention_dataset.variables)




{'uci_id': 468, 'name': 'Online Shoppers Purchasing Intention Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/468/data.csv', 'abstract': 'Of the 12,330 sessions in the dataset,\n84.5% (10,422) were negative class samples that did not\nend with shopping, and the rest (1908) were positive class\nsamples ending with shopping.', 'area': 'Business', 'tasks': ['Classification', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 12330, 'num_features': 17, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Revenue'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2018, 'last_updated': 'Thu Jan 11 2024', 'dataset_doi': '10.24432/C5F88Q', 'creators': ['C. Sakar', 'Yomi Kastro'], 'intro_paper': {'title': 'Real-time prediction of online shoppers’ purchasing intention using multi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, f1_score
import seaborn as sns
from sklearn.metrics import classification_report

# Tiền xử lý đầu vào
Dùng StandardScaler hoặc MinMaxScaler đều được, chưa thử kết hợp Pipeline.

In [ ]:
df=pd.DataFrame()

In [ ]:

df = X[['Month', 'VisitorType']].copy()

df = pd.get_dummies(df, columns=['Month'])
df = pd.get_dummies(df, columns=['VisitorType'])


In [ ]:
X = pd.get_dummies(X, columns=['Month', 'VisitorType'])

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

# Compare model by PyCaret
So sánh các model bằng compare model trong Pycaret với dữ liệu đầu vào chưa tiền xử lý, sau đó lựa chọn các model tối ưu nhất để training và lấy f1-score.

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import setup, compare_models

In [ ]:
df = X.copy()
df['Revenue'] = y

In [ ]:
model_setup = setup(data=df, target='Revenue', session_id=123,fold=5)

,Description,Value
0,Session id,123
1,Target,Revenue
2,Target type,Binary
3,Original data shape,"(12330, 29)"
4,Transformed data shape,"(12330, 29)"
5,Transformed train set shape,"(8631, 29)"
6,Transformed test set shape,"(3699, 29)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9028,0.9278,0.6018,0.7242,0.6569,0.6009,0.6046,1.2020
gbc,Gradient Boosting Classifier,0.9011,0.9292,0.6056,0.7130,0.6545,0.5972,0.6002,1.3580
rf,Random Forest Classifier,0.9009,0.9244,0.5427,0.7494,0.6291,0.5736,0.5838,0.9920
xgboost,Extreme Gradient Boosting,0.8943,0.9174,0.5719,0.6916,0.6257,0.5649,0.5685,0.4200
et,Extra Trees Classifier,0.8932,0.9158,0.4506,0.7621,0.5660,0.5097,0.5328,1.0940
ada,Ada Boost Classifier,0.8887,0.9115,0.5584,0.6678,0.6080,0.5438,0.5468,0.6780
lr,Logistic Regression,0.8843,0.8872,0.3870,0.7419,0.5081,0.4498,0.4808,2.1960
lda,Linear Discriminant Analysis,0.8795,0.8983,0.3361,0.7454,0.4629,0.4059,0.4471,0.0700
svm,SVM - Linear Kernel,0.8790,0.6047,0.3330,0.7732,0.4374,0.3868,0.4402,0.0940
ridge,Ridge Classifier,0.8723,0.8987,0.2410,0.7870,0.3686,0.3192,0.3895,0.0580


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
print(best_model)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


=> *Ở đây ta thấy LightGBM, GBC, RF là các model (f1-score cao nhất) tốt nhất, sau đó là SVM,KNN và Logistic Regression*

*=> Ta chọn 6 model trên để tiền xử lý dữ liệu đầu vào,với kfold=5 để được f1-score tốt nhất*

# Random Forest

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
for train_index, test_index in kf.split(X_scaled,y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
RFclf = RandomForestClassifier(max_depth=10, random_state=0)
RFclf.fit(X_train,y_train)


**F1-score trong Classification report**

In [ ]:
class_name=np.unique(y_train).astype(str)
class_name

In [ ]:
y_pred=RFclf.predict(X_train)
y_pred

In [ ]:
report = classification_report(y_train,y_pred,target_names=class_name)
print(report)

In [ ]:
class_name=np.unique(y_test).astype(str)
class_name

In [ ]:
y_predict=RFclf.predict(X_test)
y_predict


In [ ]:
report = classification_report(y_test,y_predict,target_names=class_name)
print(report)

**F1_score(trọng số) tính bằng hàm F1-score của skilearn**

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
cv_scores = cross_val_score(RFclf, X_scaled, y, cv=kf, scoring=scorer)

In [ ]:
mean_f1 = cv_scores.mean()
print(f"Random Forest: Mean F1-score = {mean_f1:.4f}")

# Logistic Regression

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
for train_index, test_index in kf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
LRclf = LogisticRegression(random_state=0)
LRclf.fit(X_train,y_train)

*F1-score trong classification Report*

In [ ]:
y_pred=LRclf.predict(X_train)
y_pred

In [ ]:
report = classification_report(y_train,y_pred)
print(report)

In [ ]:
y_predict=LRclf.predict(X_test)
y_predict

In [ ]:
report = classification_report(y_test,y_predict)
print(report)

**F1_score (trọng số) tính bằng hàm F1-score của skilearn**

In [ ]:
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
cv_scores = cross_val_score(LRclf, X_scaled, y, cv=kf, scoring=scorer)

In [ ]:
mean_f1 = cv_scores.mean()
print(f"Logistic Regression: Mean F1-score = {mean_f1:.4f}")

# SVM

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
for train_index, test_index in kf.split(X_scaled,y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
SVCclf = SVC(gamma='auto')
SVCclf.fit(X_train,y_train)

*F1-score trong classification Report*

In [ ]:
y_predict=SVCclf.predict(X_test)
y_predict

In [ ]:
report = classification_report(y_test,y_predict)
print(report)

In [ ]:
y_pred=SVCclf.predict(X_test)
y_pred

In [ ]:
report = classification_report(y_test,y_pred)
print(report)

**F1_score tính bằng hàm F1-score của skilearn**

In [ ]:
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
cv_scores = cross_val_score(SVCclf, X_scaled, y, cv=kf, scoring=scorer)

In [ ]:
mean_f1 = cv_scores.mean()
print(f"SVC: Mean F1-score = {mean_f1:.4f}")

# KNN

In [ ]:
neigh=KNeighborsClassifier()

**F1-score trong Classification report**

In [ ]:
for train_index, test_index in kf.split(X_scaled,y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
neigh.fit(X_train,y_train)

In [ ]:
y_predict=neigh.predict(X_train)
y_predict

In [ ]:
report = classification_report(y_train, y_predict)
print(report)

In [ ]:
y_pred=neigh.predict(X_test)
y_pred

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

**Tính f1-score cần tìm bằng hàm F1-score**

In [ ]:
cv_scores = cross_val_score(neigh, X_scaled, y, cv=kf, scoring=scorer)

In [ ]:
mean_f1 = cv_scores.mean()
print(f"KNN: Mean F1-score = {mean_f1:.4f}")

# GBM

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
for train_index, test_index in kf.split(X_scaled,y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
GBMclf = GradientBoostingClassifier()

In [ ]:
GBMclf.fit(X_train,y_train)

In [ ]:
y_predict=GBMclf.predict(X_train)
y_predict

In [ ]:
report = classification_report(y_train, y_predict)
print(report)

In [ ]:
y_pred=GBMclf.predict(X_test)
y_pred

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
cv_scores = cross_val_score(GBMclf, X_scaled, y, cv=kf, scoring=scorer)

In [ ]:
mean_f1 = cv_scores.mean()
print(f"GBM: Mean F1-score = {mean_f1:.4f}")

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
for train_index, test_index in kf.split(X_scaled,y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(X_train,y_train)

In [ ]:
y_predict=gnb.predict(X_train)
y_predict

In [ ]:
report = classification_report(y_train, y_predict)
print(report)

In [ ]:
y_pred=neigh.predict(X_test)
y_pred

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
cv_scores = cross_val_score(gnb, X_scaled, y, cv=kf, scoring=scorer)

In [ ]:
mean_f1 = cv_scores.mean()
print(f"Naive Bayes: Mean F1-score = {mean_f1:.4f}")

#Visuallize
Trực quan hóa để đưa ra cái nhìn tổng quan nhất qua f1-score của từng model

In [ ]:
models = {
    'RF': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'GBM': GradientBoostingClassifier()
}

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
results = {}
for name, model in models.items():
    cv_results = cross_val_score(model, X_scaled, y, cv=kf, scoring=scorer)
    results[name] = cv_results.mean()


In [ ]:
for name, score in results.items():
    print(f"{name}: Mean F1-score = {score:.4f}")

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(x=list(results.keys()), y=list(results.values()), palette='viridis')
plt.title('Model Comparison Based on Mean F1-Score', fontsize=16)
plt.xlabel('Models', fontsize=12)
plt.ylabel('Mean F1-Score', fontsize=12)
plt.show()

=> Giống như kết quả so sánh ban đầu, GBM,RF đều có f1_score cao nhất .